In [1]:
from gensim.models import KeyedVectors


In [2]:
model = KeyedVectors.load_word2vec_format ( "/Users/macbook/Downloads/GoogleNews-vectors-negative300.bin" , binary = True , limit = 100000 )


In [4]:
model["pianist"]


array([ 1.42578125e-01, -2.57812500e-01, -1.81640625e-01, -1.08398438e-01,
        1.01074219e-01,  4.04296875e-01, -4.59671021e-04, -1.37695312e-01,
        2.53906250e-01, -2.07031250e-01,  1.07910156e-01, -2.13867188e-01,
        2.16064453e-02,  8.25195312e-02, -4.30297852e-03,  1.25976562e-01,
        3.53515625e-01,  2.98828125e-01,  5.78613281e-02, -2.67578125e-01,
       -7.95898438e-02,  2.63671875e-01,  1.49414062e-01, -5.29785156e-02,
       -3.46679688e-02, -8.49609375e-02, -1.97753906e-02, -1.26953125e-01,
       -5.11718750e-01,  3.22265625e-01, -2.11914062e-01, -5.83496094e-02,
       -2.83203125e-01, -6.43920898e-03,  3.67187500e-01, -5.50781250e-01,
        2.04101562e-01,  2.73437500e-01,  1.05468750e-01,  1.52587891e-03,
       -1.97265625e-01, -8.44726562e-02,  1.25976562e-01,  1.05957031e-01,
        4.31640625e-01, -3.75976562e-02,  9.57031250e-02,  2.05078125e-02,
        1.72851562e-01,  2.07031250e-01,  3.66210938e-02,  1.03027344e-01,
       -2.49023438e-01,  

In [5]:
model.most_similar("pianist")


[('violinist', 0.8616065382957458),
 ('cellist', 0.835774302482605),
 ('pianists', 0.8017312288284302),
 ('jazz_pianist', 0.7941479086875916),
 ('saxophonist', 0.7860057353973389),
 ('clarinetist', 0.7802671194076538),
 ('soloist', 0.7775945663452148),
 ('trombonist', 0.7416003942489624),
 ('trumpeter', 0.7403372526168823),
 ('piano', 0.7373092174530029)]

In [8]:
model.most_similar(positive=["pianist", "cello"], negative=["piano"])


[('cellist', 0.8336730599403381),
 ('violinist', 0.8203256726264954),
 ('soloist', 0.7550286650657654),
 ('clarinetist', 0.7097249031066895),
 ('viola', 0.7015129923820496),
 ('violinists', 0.6962666511535645),
 ('pianists', 0.6920492649078369),
 ('flutist', 0.6915562152862549),
 ('soprano', 0.6902468204498291),
 ('soloists', 0.6888906955718994)]

In [15]:
model.most_similar(positive=["clock", "temperature"], negative=["time"])



[('temperatures', 0.5541223287582397),
 ('thermometer', 0.5271066427230835),
 ('Temperature', 0.513730525970459),
 ('thermostat', 0.4789181351661682),
 ('degrees_Celsius', 0.4706636071205139),
 ('degrees_Fahrenheit', 0.4640905261039734),
 ('°_C', 0.4619429111480713),
 ('thermometers', 0.45511335134506226),
 ('humidity', 0.44329598546028137),
 ('°_F', 0.4413301348686218)]